## 머신러닝 실습

### 텍스트 마이닝

#### 데이터 수집

##### 네이버 코로나 뉴스로 감성분석

In [ ]:
# 필요 라이브러리 등록
import json

In [ ]:
with open('./data/코로나_naver_news.json', encoding = 'utf-8') as f:
    data = json.load(f)

In [ ]:
import pandas as pd
import re

In [ ]:
dfData = pd.DataFrame(data)

In [ ]:
dfData.head()

In [ ]:
## 한글 이외의 것 다 제거하는 작업
dfData['title'] = dfData['title'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+',' ', x))

In [ ]:
dfData['description'] = dfData['description'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+',' ', x))

In [ ]:
dfData[['title','pDate','description']].to_excel('./data/코로나뉴스_전처리.xlsx',index = False)

In [ ]:
dfData = pd.read_excel('./data/코로나뉴스_전처리.xlsx')

In [ ]:
dfData.head()

##### 감성분석 모델 재구축

In [ ]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('../day12/data/ratings_train.txt',engine='python', sep='\t', encoding='utf-8')

In [20]:
dfNsmcTrain.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [ ]:
## 한글이외에 다 제거
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+', ' ' , x))